In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datastorm-data/final_val.csv
/kaggle/input/datastorm-data/final_cleaned.csv
/kaggle/input/test-set/test_kaggle.csv
/kaggle/input/cleaned/cleaned.csv
/kaggle/input/cleaned-data/cleaned (1).csv


In [2]:
train_set = pd.read_csv('/kaggle/input/datastorm-data/final_cleaned.csv')
val_set = pd.read_csv('/kaggle/input/datastorm-data/final_val.csv')

In [3]:
clean_set = pd.read_csv('/kaggle/input/cleaned-data/cleaned (1).csv')

In [4]:
train_set = train_set.drop(columns=train_set.columns[0])
val_set = val_set.drop(columns=val_set.columns[0])

In [5]:
label_mapping = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}

In [6]:
X_train = train_set.drop(columns=['category'])
y_train = train_set['category']

X_val = val_set.drop(columns=['cluster_catgeory'])
y_val = val_set['cluster_catgeory']

In [7]:
# label rearrenge
y_val_mod = y_val.replace(label_mapping)
y_train_mod = y_train.replace(label_mapping)

In [8]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [9]:
# Create DMatrix
train_data = xgb.DMatrix(X_train, label=y_train_mod)
val_data = xgb.DMatrix(X_val, label=y_val_mod)

In [10]:
X_train

,luxury_sales,fresh_sales,dry_sales
0,0.1327,0.1735,0.6938
1,0.1183,0.1720,0.7097
2,0.1300,0.1800,0.6900
3,0.1515,0.1717,0.6768
4,0.1354,0.1562,0.7084
...,...,...,...
861848,0.1720,0.7205,0.1075
861849,0.1895,0.6842,0.1263
861850,0.1915,0.7021,0.1064
861851,0.1717,0.6869,0.1414


In [11]:
params = {
    'objective': 'multi:softprob',  # Objective for multi-class classification
    'num_class': 6,  # Number of classes
    'eval_metric': 'mlogloss',  # Metric for multi-class classification
    'max_depth': 5,
    'eta': 0.5, 
    'missing': None,
    'random_state':42,
    'tree_method':'hist',
    'subsample':0.8,
    'colsample_bytree':1,
    'verbosity':2   
}

# Train the model
model = xgb.train(params, train_data, num_boost_round=1000, early_stopping_rounds=50, evals=[(val_data, 'val')], verbose_eval=100)

[0]	val-mlogloss:0.68873
[74]	val-mlogloss:0.08929


In [12]:
# Make predictions
y_pred_class = model.predict(xgb.DMatrix(X_val))
y_pred_class = np.argmax(y_pred_class, axis=1)  # Get class labels

# Evaluate
accuracy = accuracy_score(y_val_mod, y_pred_class)
print(f'Validation Accuracy: {accuracy:.4f}')

Validation Accuracy: 0.9371


In [13]:
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [14]:
import lightgbm as lgb

# Create dataset
train_data = lgb.Dataset(X_train, label=y_train_mod)
val_data = lgb.Dataset(X_val, label=y_val_mod)

# Set parameters
params = {
    'objective': 'multiclass',
    'num_class': 6,  # Number of classes
    'metric': 'multi_logloss',  # Metric for multi-class classification
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42
}

lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 6,
        'metric': 'multi_logloss',
        'learning_rate': 0.2,
        #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
        'num_leaves': 31,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 5,  # -1 means no limit
        'min_child_samples': 20,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 255,  # Number of bucketed bin for feature values
        'subsample': 0.6,  # Subsample ratio of the training instance.
        'subsample_freq': 0,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.3,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 5,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'reg_alpha': 0,  # L1 regularization term on weights
        'reg_lambda': 0,  # L2 regularization term on weights
        'nthread': 4,
        'verbose': 0,
    }


# Train the model
model = lgb.train(lgb_params, train_data, valid_sets=[val_data])

# Make predictions
y_pred = model.predict(X_val, num_iteration=model.best_iteration)
y_pred_class = np.argmax(y_pred, axis=1)  # Get class labels

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

In [15]:
# Evaluate the final model
accuracy = accuracy_score(y_val_mod, y_pred_class)
f1 = f1_score(y_val_mod, y_pred_class, average='macro')
print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation F1-score (macro): {f1:.4f}')
micro_prec = precision_score(y_val_mod, y_pred_class, average='micro')
print(f"Micro-averaged Precision: {micro_prec:.4f}")

Validation Accuracy: 0.9373
Validation F1-score (macro): 0.8291
Micro-averaged Precision: 0.9373


In [16]:
len(np.unique(y_train))

6

In [17]:
X_train.columns

Index(['luxury_sales', 'fresh_sales', 'dry_sales'], dtype='object')

In [18]:
X_train_ground = X_train.copy()

In [19]:
X_train_ground['luxury_sales'] = X_train['luxury_sales'].round(2)
X_train_ground['fresh_sales'] = X_train['fresh_sales'].round(2)
X_train_ground['dry_sales'] = X_train['dry_sales'].round(2)

In [20]:
X_val_ground = X_val.copy()

In [21]:
X_val_ground['luxury_sales'] = X_val['luxury_sales'].round(2)
X_val_ground['fresh_sales'] = X_val['fresh_sales'].round(2)
X_val_ground['dry_sales'] = X_val['dry_sales'].round(2)

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


one_hot_encoded_train = tf.keras.utils.to_categorical(y_train_mod)
one_hot_encoded_test = tf.keras.utils.to_categorical(y_val_mod)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y_train))

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_ground.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model.fit(X_train_ground, one_hot_encoded_train, epochs=100, batch_size=32, validation_data=(X_val_ground, one_hot_encoded_test), callbacks=[early_stop], verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_ground, one_hot_encoded_test, verbose=0)
print(f'Validation Accuracy: {accuracy:.4f}')

2024-05-17 06:27:16.842144: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 06:27:16.842395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 06:27:17.018799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
26933/26933 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step - accuracy: 0.7603 - loss: 0.4030 - val_accuracy: 0.9439 - val_loss: 0.0926
Epoch 2/100
26933/26933 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - accuracy: 0.8185 - loss: 0.2669 - val_accuracy: 0.9437 - val_loss: 0.0921
Epoch 3/100
26933/26933 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 0.8186 - loss: 0.2641 - val_accuracy: 0.9439 - val_loss: 0.0975
Epoch 4/100
26933/26933 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - accuracy: 0.8187 - loss: 0.2641 - val_accuracy: 0.9315 - val_loss: 0.0960
Epoch 5/100
26933/26933 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 0.8191 - loss: 0.2619 - val_accuracy: 0.9375 - val_loss: 0.1071
Epoch 6/100
26933/26933 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 0.8188 - loss: 0.2610 - val_accuracy: 0.9311 - val_loss: 0.0965
Epoch 7/100
26933/26933 ━━━━━━━━━━━━━━━━━━━━ 83s 2ms/step - accuracy: 0.8191 - loss: 0.2616 - val_accuracy: 0.9310 - val_loss: 0.1044
Epoch 8/100
26933/26933 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - ac

In [23]:
clean_set = pd.read_csv('/kaggle/input/cleaned-data/cleaned (1).csv')

In [24]:
clean_set.head()

,Unnamed: 0,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales,cluster_catgeory,total_sales,cluster_city_enco,luxery_sales_ratio,fresh_sales_ratio,dry_sales_ratio
0,131072,10370993.0,Moratuwa,2072.47,1462.92,8046.06,4.0,11581.45,3,0.18,0.13,0.69
1,131073,10751334.0,Nuwara Eliya,1500.60,1200.48,6502.60,4.0,9203.68,4,0.16,0.13,0.71
2,131074,10482183.0,Dehiwala-Mount Lavinia,2862.00,1431.00,9444.60,4.0,13737.60,2,0.21,0.10,0.69
3,131075,10594832.0,Kelaniya,2103.15,1822.73,9394.07,4.0,13319.95,2,0.16,0.14,0.71
4,131076,10418774.0,Peliyagoda,1115.46,619.70,4275.93,4.0,6011.09,2,0.19,0.10,0.71


In [25]:
clean_set = clean_set.drop(columns=['Customer_ID', 'luxury_sales', 'fresh_sales', 'total_sales', 'cluster_city_enco', 'dry_sales'])

In [26]:
clean_set = clean_set.drop(columns=[clean_set.columns[0]])

In [27]:
clean_set.head()

,outlet_city,cluster_catgeory,luxery_sales_ratio,fresh_sales_ratio,dry_sales_ratio
0,Moratuwa,4.0,0.18,0.13,0.69
1,Nuwara Eliya,4.0,0.16,0.13,0.71
2,Dehiwala-Mount Lavinia,4.0,0.21,0.10,0.69
3,Kelaniya,4.0,0.16,0.14,0.71
4,Peliyagoda,4.0,0.19,0.10,0.71


In [28]:
clean_set['cluster_catgeory'] = clean_set['cluster_catgeory'].astype(int)

In [29]:
label_mapping = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}

clean_set['cluster_catgeory'] = clean_set['cluster_catgeory'].replace(label_mapping)

In [30]:
clean_set.head()

,outlet_city,cluster_catgeory,luxery_sales_ratio,fresh_sales_ratio,dry_sales_ratio
0,Moratuwa,3,0.18,0.13,0.69
1,Nuwara Eliya,3,0.16,0.13,0.71
2,Dehiwala-Mount Lavinia,3,0.21,0.10,0.69
3,Kelaniya,3,0.16,0.14,0.71
4,Peliyagoda,3,0.19,0.10,0.71


In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X = clean_set.drop(columns=['cluster_catgeory'])
y = clean_set['cluster_catgeory']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm



# Identify categorical features
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'object']

# Convert categorical features to category data type
for feature in categorical_features:
    X_train[feature] = X_train[feature].astype('category')
    X_val[feature] = X_val[feature].astype('category')

# Define hyperparameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'max_depth': -1,
    'random_state': 42
}

# Train the model with early stopping and optimized hyperparameters
model = LGBMRegressor(**params)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])


# Make predictions on the validation set
val_preds = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, val_preds)
print(f'Validation MSE: {mse:.4f}')

Validation MSE: 0.0011


In [34]:
val_preds = np.round(val_preds).astype(int)

In [35]:
# Evaluate the final model
accuracy = accuracy_score(y_val, val_preds)
f1 = f1_score(y_val, val_preds, average='macro')
print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation F1-score (macro): {f1:.4f}')
micro_prec = precision_score(y_val, val_preds, average='micro')
print(f"Micro-averaged Precision: {micro_prec:.4f}")

Validation Accuracy: 0.9998
Validation F1-score (macro): 0.9997
Micro-averaged Precision: 0.9998


In [36]:
from catboost import CatBoostClassifier, Pool



# Identify categorical features
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'object']

# Create Pool objects
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)

# Set parameters
params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'depth': 6,
    'l2_leaf_reg': 3,
    'random_strength': 1,
    'bagging_temperature': 0.5,
    'metric_period': 50,
    'od_type': 'Iter',
    'od_wait': 50,
    'allow_writing_files': False
}

# Train the model
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=val_pool, use_best_model=True, verbose=100)

# Make predictions on the validation set
val_preds = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, val_preds)
print(f'Validation MSE: {mse:.4f}')

# Evaluate the final model
accuracy = accuracy_score(y_val, val_preds)
f1 = f1_score(y_val, val_preds, average='macro')
print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation F1-score (macro): {f1:.4f}')
micro_prec = precision_score(y_val, val_preds, average='micro')
print(f"Micro-averaged Precision: {micro_prec:.4f}")

0:	learn: 1.3529215	test: 1.3528612	best: 1.3528612 (0)	total: 1.39s	remaining: 23m 13s
100:	learn: 0.0018878	test: 0.0018037	best: 0.0018030 (98)	total: 1m 33s	remaining: 13m 48s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.001794152772
bestIteration = 121

Shrink model to first 122 iterations.
Validation MSE: 0.0011
Validation Accuracy: 0.9998
Validation F1-score (macro): 0.9997
Micro-averaged Precision: 0.9998


In [37]:
X_train_np = X_train.copy()
X_val_np = X_val.copy()

In [38]:
X_train_np = X_train.drop(columns=['outlet_city'])
X_val_np = X_val.drop(columns=['outlet_city'])

In [39]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


one_hot_encoded_train = tf.keras.utils.to_categorical(y_train)
one_hot_encoded_test = tf.keras.utils.to_categorical(y_val)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y_train))

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_np.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model.fit(X_train_np, one_hot_encoded_train, epochs=15, batch_size=32, validation_data=(X_val_np, one_hot_encoded_test), callbacks=[early_stop], verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_np, one_hot_encoded_test, verbose=0)
print(f'Validation Accuracy: {accuracy:.4f}')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
15303/15303 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.8781 - loss: 0.2714 - val_accuracy: 0.9534 - val_loss: 0.0779
Epoch 2/15
15303/15303 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9481 - loss: 0.0954 - val_accuracy: 0.9509 - val_loss: 0.0930
Epoch 3/15
15303/15303 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9495 - loss: 0.0901 - val_accuracy: 0.9542 - val_loss: 0.0759
Epoch 4/15
15303/15303 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9499 - loss: 0.0870 - val_accuracy: 0.9540 - val_loss: 0.0775
Epoch 5/15
15303/15303 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9510 - loss: 0.0856 - val_accuracy: 0.9525 - val_loss: 0.0775
Epoch 6/15
15303/15303 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9507 - loss: 0.0853 - val_accuracy: 0.9542 - val_loss: 0.0830
Epoch 7/15
15303/15303 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9508 - loss: 0.0854 - val_accuracy: 0.9516 - val_loss: 0.0798
Epoch 8/15
15303/15303 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 

In [40]:
test = pd.read_csv('/kaggle/input/test-set/test_kaggle.csv')

In [41]:
def convert_to_float_and_find_errors(dataframe, column):
    error_rows = []
    for index, value in dataframe[column].items():
        try:
            float(value)
        except ValueError:
            error_rows.append((index, value))
    return error_rows

# Convert 'luxury_sales' column to float and find rows causing ValueError
error_rows = convert_to_float_and_find_errors(test, 'luxury_sales')

# Print rows causing ValueError
print("Rows causing ValueError:")
for index, value in error_rows:
    print(f"Index: {index}, Value: {value}")


Rows causing ValueError:
Index: 6, Value: Seven Hundread and sixty Ruppees
Index: 6299, Value: Eight Hundread and fifteen ruppes


In [42]:
test['luxury_sales'][6] = 760.0 
test['luxury_sales'][6299] = 850.0
test['fresh_sales'][13949] = 9000.0
test['dry_sales'][40746] = 3700.0

/tmp/ipykernel_18/1883175211.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test['luxury_sales'][6] = 760.0
/tmp/ipykernel_18/1883175211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

In [43]:
test['luxury_sales'] = test['luxury_sales'].astype(float)
test['fresh_sales'] = test['fresh_sales'].astype(float)
test['dry_sales'] = test['dry_sales'].astype(float)

In [44]:
test['total_sales'] = test['luxury_sales'] + test['fresh_sales'] + test['dry_sales']
test['total_sales'].describe().round(2)

count    40749.00
mean     11061.55
std       3832.32
min       4603.68
25%       7861.56
50%      10801.00
75%      13748.40
max      20784.40
Name: total_sales, dtype: float64

In [45]:
test['luxury_sales'] = (test['luxury_sales'] / test['total_sales']).round(4)  # Round to four decimal places
test['fresh_sales'] = (test['fresh_sales'] / test['total_sales']).round(4)
test['dry_sales'] = (test['dry_sales'] / test['total_sales']).round(4)

# Normalize the ratios so that the sum adds up to 1
total_ratio = test['luxury_sales'] + test['fresh_sales'] + test['dry_sales']
test['luxury_sales'] /= total_ratio
test['fresh_sales'] /= total_ratio
test['dry_sales'] /= total_ratio

In [46]:
test['luxury_sales'] = test['luxury_sales'].round(4)  # Round to four decimal places
test['fresh_sales'] = test['fresh_sales'].round(4)
test['dry_sales'] = test['dry_sales'].round(4)

In [47]:
test.head()

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales,total_sales
0,33574,batticaloa,0.1429,0.1905,0.6666,18805.50
1,10089,batticaloa,0.1250,0.1875,0.6875,13740.48
2,38329,batticaloa,0.1170,0.1702,0.7128,7298.16
3,11376,batticaloa,0.1237,0.1753,0.7010,13241.47
4,12410,batticaloa,0.1300,0.1900,0.6800,7993.00


In [48]:
Customer_ID = test[['Customer_ID']]

In [49]:
X = test.drop(columns=['Customer_ID', 'total_sales', 'outlet_city'])

In [50]:
submit_data = xgb.DMatrix(X)

In [51]:
# Make predictions
y_pred_class = model.predict(X)

1274/1274 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [52]:
y_pred_class

array([[9.99999464e-01, 1.71964438e-15, 1.43749822e-20, 4.49093790e-07,
        1.35745407e-21, 1.14621636e-30],
       [9.99999940e-01, 3.25438082e-19, 6.69030592e-26, 1.44846179e-08,
        3.16552375e-27, 2.24094109e-38],
       [9.99999940e-01, 1.72384339e-21, 8.76820052e-29, 3.05798142e-09,
        2.87956713e-30, 0.00000000e+00],
       ...,
       [6.93785466e-07, 1.47407878e-10, 4.16574836e-01, 9.92778732e-05,
        5.83323538e-01, 1.71710440e-06],
       [5.48468108e-07, 9.35948541e-11, 4.15561378e-01, 9.01541498e-05,
        5.84346592e-01, 1.35323080e-06],
       [7.17339049e-07, 1.54152829e-10, 4.16739047e-01, 1.01497215e-04,
        5.83157003e-01, 1.74732065e-06]], dtype=float32)

In [53]:
labels = np.argmax(y_pred_class, axis=1) + 1

In [54]:
labels

array([1, 1, 1, ..., 5, 5, 5])

In [55]:
array_df = pd.DataFrame({"cluster category": labels})

In [56]:
array_df['cluster category'].unique()

array([1, 6, 2, 5, 4])

In [57]:
label_unmapping = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6}

array_df['cluster category'] = array_df['cluster category'].replace(label_unmapping)

In [58]:
array_df.head()

,cluster category
0,2
1,2
2,2
3,2
4,2


In [59]:
submission_1 = pd.concat([Customer_ID, array_df], axis=1)


In [60]:
submission_1.to_csv('ann_model_out.csv', index=False)